In [1]:
# !pip install oolearning --upgrade

In [13]:
import copy
import os
import oolearning as oo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from helpers import column_log, BinaryAucRocScore

pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
width = 10
plt.rcParams['figure.figsize'] = [width, width/1.333]

In [14]:
csv_file = '../census.csv'
target_variable = 'income'

explore = oo.ExploreClassificationDataset.from_csv(csv_file_path=csv_file,
                                                   target_variable=target_variable)
negative_class = '<=50K'
positive_class = '>50K'

explore.dataset.head(20)

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K
5,37,Private,Masters,14.0,Married-civ-spouse,Exec-managerial,Wife,White,Female,0.0,0.0,40.0,United-States,<=50K
6,49,Private,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0.0,0.0,16.0,Jamaica,<=50K
7,52,Self-emp-not-inc,HS-grad,9.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,45.0,United-States,>50K
8,31,Private,Masters,14.0,Never-married,Prof-specialty,Not-in-family,White,Female,14084.0,0.0,50.0,United-States,>50K
9,42,Private,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178.0,0.0,40.0,United-States,>50K


In [15]:
global_transformations = [oo.ImputationTransformer(),
                          oo.StatelessColumnTransformer(columns=['capital-gain', 'capital-loss'],
                                                        custom_function=column_log),
                          oo.CenterScaleTransformer(),
                          oo.DummyEncodeTransformer(oo.CategoricalEncoding.ONE_HOT)]

In [16]:
params_dict = dict(
    max_depth=[30, 60, 90],
    max_features=[0.4, 0.65, 0.9],
    min_samples_leaf=[2],
    min_samples_split=[2, 9, 16],
)
grid = oo.HyperParamsGrid(params_dict=params_dict)
grid.params_grid

,max_depth,max_features,min_samples_leaf,min_samples_split
0,30,0.40,2,2
1,30,0.40,2,9
2,30,0.40,2,16
3,30,0.65,2,2
4,30,0.65,2,9
5,30,0.65,2,16
6,30,0.90,2,2
7,30,0.90,2,9
8,30,0.90,2,16
9,60,0.40,2,2


In [17]:
hyper_param_object = oo.RandomForestHP(
    criterion='gini',
    num_features=None,
    max_features=0.4087148680294206,
    n_estimators=1815,
    max_depth=96,
    min_samples_split=16,
    min_samples_leaf=2,
    min_weight_fraction_leaf=0.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0,
    )

In [18]:
model=oo.RandomForestClassifier()

In [19]:
repeats = 5
folds = 5

In [20]:
model_cache_directory = 'tuner_RandomForest'
resampler_cache_directory = 'tuner_RandomForest_resample_cache'

In [21]:
score_list = [oo.AucRocScore(positive_class=positive_class),
              oo.FBetaScore(converter=oo.TwoClassThresholdConverter(threshold=0.5,
                                                                    positive_class=positive_class),
                            beta=0.5),
              oo.SensitivityScore(converter=oo.TwoClassThresholdConverter(threshold=0.5,
                                                                          positive_class=positive_class)),
              oo.PositivePredictiveValueScore(converter=oo.TwoClassThresholdConverter(threshold=0.5,
                                                                                      positive_class=positive_class))]

In [22]:

# define/configure the resampler
resampler = oo.RepeatedCrossValidationResampler(model=model,  # using a Random Forest model
                                             transformations=global_transformations,
                                             scores=[s.clone() for s in score_list],
                                             folds=folds,
                                             repeats=repeats)
# define/configure the ModelTuner
tuner = oo.ModelTuner(resampler=resampler,
                      hyper_param_object=hyper_param_object,
                      model_persistence_manager=oo.LocalCacheManager(cache_directory=model_cache_directory),
                      resampler_persistence_manager=oo.LocalCacheManager(cache_directory=resampler_cache_directory),
                      #parallelization_cores=-1,
                      parallelization_cores=0,
                     )

In [ ]:
tuner.tune(data_x=explore.dataset.drop(columns=target_variable),
           data_y=explore.dataset[target_variable],
           params_grid=grid)

In [ ]:
tuner.results.best_hyper_params

In [ ]:
tuner.results.best_model_resampler_object.score_means

In [ ]:
tuner.results.plot_resampled_stats()

In [ ]:
tuner.results.plot_resampled_scores(metric=oo.Metric.AUC_ROC,
                                    x_axis_limits=(0.86, 0.89),
                                    show_one_ste_rule=True
                                   )

In [ ]:
tuner.results.plot_resampled_scores(oo.Metric.FBETA_SCORE,
                                    x_axis_limits=(0.64, 0.67),
                                    show_one_ste_rule=True
                                   )

In [ ]:
params_dict

In [ ]:
tuner.results.plot_hyper_params_profile(metric=oo.Metric.AUC_ROC,
                                        x_axis='reg_alpha',
                                        line='reg_lambda',)

In [ ]:
tuner.results.plot_hyper_params_profile(metric=oo.Metric.AUC_ROC,
                                        x_axis='al',
                                        line='min_child_weight',
                                        grid='n_estimators')

In [ ]:
tuner.results.plot_hyper_params_profile(metric=oo.Metric.AUC_ROC,
                                        x_axis='max_depth',
                                        line='min_child_weight',
                                        grid='subsample')

In [ ]:
assert False

# Resample Ideal Threshold

In [ ]:
final_hyper_param_object = oo.XGBoostTreeHP(
                                    objective=oo.XGBObjective.BINARY_LOGISTIC,
                                    learning_rate=0.05,
                                    n_estimators=2000,
                                    max_depth=3,
                                    min_child_weight=2,
                                    gamma=0.1,
                                    subsample=0.95,
                                    colsample_bytree=0.15,
                                    reg_alpha=0.01,
                                    reg_lambda=2,
                                    scale_pos_weight=scale_pos_weight_calc,)

final_model = oo.XGBoostClassifier()
final_transformations = [
                         # kaggle test file has white space around values
                         oo.StatelessColumnTransformer(columns=explore.categoric_features,
                                                       custom_function=lambda x: x.str.strip()),
                         oo.ImputationTransformer(),
                         oo.StatelessColumnTransformer(columns=['capital-gain', 'capital-loss'],
                                                       custom_function=column_log),
                         oo.CenterScaleTransformer(),
                         oo.DummyEncodeTransformer(oo.CategoricalEncoding.ONE_HOT)
]

In [ ]:
decorator = oo.TwoClassThresholdDecorator(parallelization_cores=0)
resampler = oo.RepeatedCrossValidationResampler(
    model=final_model.clone(),
    transformations=[t.clone() for t in final_transformations],
    scores=[s.clone() for s in score_list],
    folds=5,
    repeats=2,
    fold_decorators=[decorator])
resampler.resample(data_x=explore.dataset.drop(columns=target_variable),
                   data_y=explore.dataset[target_variable],
                   hyper_params=final_hyper_param_object.clone())

In [ ]:
resampler.results.plot_resampled_scores()

In [ ]:
decorator.roc_ideal_thresholds

In [ ]:
np.mean(decorator.roc_ideal_thresholds)

In [ ]:
np.median(decorator.roc_ideal_thresholds)

In [ ]:
ideal_threshold = np.mean(decorator.roc_ideal_thresholds)
ideal_threshold

In [ ]:
# use the ideal threshold for the evaluator in order to view ROC
evaluator = oo.TwoClassProbabilityEvaluator(converter=
                    oo.TwoClassThresholdConverter(threshold=ideal_threshold,  # ideal threshold
                                                  positive_class=positive_class))

trainer = oo.ModelTrainer(model=final_model.clone(),
                          model_transformations=[t.clone() for t in final_transformations],
                          splitter=None,  # don't split, train on all data
                          evaluator=evaluator,
                          scores=[s.clone() for s in score_list])
trainer.train(data=explore.dataset, target_variable=target_variable, hyper_params=final_hyper_param_object.clone())

trainer.training_evaluator.all_quality_metrics

In [ ]:
trainer.training_scores[0].value

In [ ]:
trainer.training_evaluator.plot_roc_curve()

In [ ]:
csv_file = '../test_census.csv'
test_dataset = pd.read_csv(csv_file)
test_dataset.shape

In [ ]:
test_dataset.head(100)

In [ ]:
indexes = test_dataset['Unnamed: 0']

predictions = trainer.predict(test_dataset.drop(columns='Unnamed: 0'))
test_converter = oo.TwoClassThresholdConverter(threshold=ideal_threshold,  # ideal thresholds
                                               positive_class=positive_class)
predictions.head()

In [ ]:
class_predictions = test_converter.convert(predictions)
class_predictions[0:5]

In [ ]:
# previous_predictions = pd.read_csv('submission_5_xgb.csv')
# previous_predictions.head(10)

In [ ]:
income_value = predictions['>50K']#[0 if x == '<=50K' else 1 for x in class_predictions]
new_predictions = pd.DataFrame({'id': indexes, 'income': income_value})
new_predictions.to_csv('submission_7_xgb.csv', index=False)

In [ ]:
# # e.g. lower threshold will change 0's to 1's so new - previous will give number predictions that changed to 1
# difference = new_predictions.income - previous_predictions.income
# difference.sum() / len(difference)

# Results

## 1)

## 2)

## 3)